In [1]:
from IPython.display import display
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error
from scipy.stats import zscore
import gzip
import os
import csv
from alive_progress import alive_bar
from collections import defaultdict
import time


c:\Users\bincl\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\__init__.py:169: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col, element_at, slice, size, regexp_extract, transform, when, explode, \
monotonically_increasing_id, map_from_arrays, lit, udf,collect_list, row_number, ceil, map_keys, expr, from_json
from pyspark.sql.types import ShortType, ArrayType, LongType, StringType
from pyspark.sql import types as T
from pyspark.conf import SparkConf
import pandas as pd

#path = 'C:/Users/bincl/BA-Thesis/Dataset/2gram/2_20000_nopos_ab.gz'
start_date = 1800
end_date = 2000
directory_2gram = 'C:/Users/bincl/BA-Thesis/Dataset/2gram/parquet/'
directory_1gram = 'C:/Users/bincl/BA-Thesis/Dataset/1gram/parquet/'


In [3]:
#spark = SparkSession.builder.config(conf=conf).appName('NgramSQL').getOrCreate()
spark = SparkSession.builder.appName('NgramSQL').getOrCreate()
df_2gram = spark.read.option("header","true").option("recursiveFileLookup","true").parquet(directory_2gram)  

df_1gram = spark.read.option("header","true").option("recursiveFileLookup","true").parquet(directory_1gram).cache()   

In [4]:
# if id set as default parameter then a list is given if not a spark dataframe is given as parameter df
def get_pd_df(df):
    matched = df['Data'] 
    matched_pandas_df = pd.Series(matched)
    matched_pandas_df.sort_index(inplace = True)

    matched_pandas_df.index.astype('int')
    numbers = list(range(start_date,end_date))
    numbers = map(str, numbers)
    matched_pandas_df = matched_pandas_df.reindex(numbers, fill_value= 0)
    return matched_pandas_df

def get_pd_df_from_string(suchstring, df):
    x = df.select(['Tokens', 'Data']).where(df.Tokens == suchstring)
    matched = x.first()['Data']
    matched_pandas_df = pd.Series(matched)
    matched_pandas_df.sort_index(inplace = True)

    matched_pandas_df.index.astype('int')
    numbers = list(range(start_date,end_date))
    numbers = map(str, numbers)
    matched_pandas_df = matched_pandas_df.reindex(numbers, fill_value= 0)
    return matched_pandas_df

In [5]:
# deprecated zu langsam
def find_n_gram(direc, find):
    for filename in os.listdir(direc):
        f = os.path.join(direc, filename)
        # checking if it is a file
        if os.path.isfile(f):
            with gzip.open(f,'rt', encoding='utf-8') as input:
                str_arr_csv = input.readlines()
                for line in str_arr_csv:
                    token = line.split("\t")
                    if find == token[0]:
                        return line
                        
    return "line not found" 

def start_the_search(directory,search):
    result = find_n_gram(directory,search)
    
    if result == "line not found":
        print(result)
        exit()
    result = result.split("\t")
    lower_ngram = pd.Series(result[1:],dtype= "string", name = result[0])
    return lower_ngram

In [6]:
def getLowerLevelString(string):
   first, *middle, last = string.split()
   if len(middle) != 0:
        first = first + " " + ''.join(middle)
        last = ''.join(middle) + " " + last    
   return first, last

def MLR(token,data):
    y = data
    first,last = getLowerLevelString(token)
    first = get_pd_df_from_string(first,df_1gram)
    last = get_pd_df_from_string(last,df_1gram)
    X = pd.concat([first,last],axis=1)
    reg = LinearRegression()
    reg.fit(X, y) 
    return reg.coef_,reg.intercept_,X,y

def buildApproximation(c1,c2,basevalue,df):
    df = df.astype('float')
    
    df = df.fillna(0)
    
    df['scaledFirst'] = df.iloc[:,0].apply(lambda x: x * c1) 
    df['scaledLast'] = df.iloc[:,1].apply(lambda x: x * c2) 
    df['approximation'] = df['scaledFirst'] + df['scaledLast'] + basevalue
    return df

In [7]:
def compressWithError2gram(n,error):
    df_2_gram = df_2gram.head(n)
    result = []
    all = []
    sum = []
    firstN = range(0,n)
    
    with alive_bar(len(firstN), force_tty = True, bar = 'smooth') as bar:
        for i in firstN:
            df_file = df_2_gram[i]
            data = get_pd_df(df_file)
            token = df_file['Tokens']
            if not(data.eq(0).all()):
                coef,intercept,dfAprox,original = MLR(token,data)
                c1,c2 = coef
                df = buildApproximation(c1,c2,intercept,dfAprox)
                dfOriginal = pd.DataFrame()
                dfOriginal['values'] = original
                dfOriginal['values'] = dfOriginal['values'].apply(pd.to_numeric, errors='coerce')
                #dfOriginal= pd.to_numeric(dfOriginal)
                dfOriginal['zscore'] = zscore(dfOriginal)
                df['zscore'] = zscore(df['approximation'])
                sum.append(pd.to_numeric(dfOriginal['values']).sum()) 
                #sum.append(df_file['Sum']) 
                rmse = mean_squared_error(dfOriginal['zscore'], df['zscore'], squared = False)
                #rmse = mean_squared_error(dfOriginal, df['approximation'])
                if rmse <= error:
                    result.append([token,rmse,dfOriginal['values'],dfOriginal['zscore'],df['approximation'],df['zscore']]) 
                all.append(rmse)    
            bar()
    return result, all, sum


In [8]:
result, all, sum = compressWithError2gram(100000, 1)
print(sum)
print(all)
print(len(result))

|▎⚠︎                                      | (!) 665/100000 [1%] in 1:04.0 (10.40/▅▇▇ 146/100000 [0%] in 17s (~3:09:29, ▆▄▂ 151/100000 [0%] in 17s (~3:08:22, ▅▇▇ 202/100000 [0%] in 22s (~2:59:54, ▃▁▃ 224/100000 [0%] in 24s (~2:57:21, ▄▂▂ 252/100000 [0%] in 26s (~2:54:32, ▁▃▅ 256/100000 [0%] in 27s (~2:53:58, ▅▇▇ 289/100000 [0%] in 30s (~2:50:58, ▇▇▅ 306/100000 [0%] in 31s (~2:49:37, ▃▅▇ 316/100000 [0%] in 32s (~2:49:04, ▅▃▁ 324/100000 [0%] in 33s (~2:48:42, ▄▂▂ 326/100000 [0%] in 33s (~2:48:35, ▅▇▇ 347/100000 [0%] in 35s (~2:47:20, ▅▇▇ 362/100000 [0%] in 36s (~2:46:24, ▂▄▆ 402/100000 [0%] in 40s (~2:44:56, █▆▄ 437/100000 [0%] in 43s (~2:43:41, ▅▇▇ 489/100000 [0%] in 48s (~2:42:46, ▄▆█ 515/100000 [1%] in 51s (~2:42:49, ▇▇▅ 547/100000 [1%] in 53s (~2:42:02, ▃▁▃ 554/100000 [1%] in 54s (~2:41:51, ▅▃▁ 566/100000 [1%] in 55s (~2:41:32, █▆▄ 637/100000 [1%] in 1:01 (~2:39:46


KeyboardInterrupt: 

In [ ]:
plt.boxplot(all)
plt.show()
rmse_with_error = []

for i in result:
    rmse_with_error.append(i[1])
plt.boxplot(rmse_with_error)
plt.show()

import seaborn as sns

sns.violinplot(x= rmse_with_error, inner="point")

In [ ]:
n = 0
result[n][2].plot()
result[n][4].plot()
plt.show()
result[n][3].plot()
result[n][5].plot()
plt.show()
print(result[n][0])
print(result[n][1])


In [ ]:
sns.violinplot(x= all, inner_kws=dict(box_width=15, whis_width=2, color=".8"))

In [ ]:

plt.scatter(sum, all)
plt.xscale('log')
plt.show()
print("2gram test 30 min für 10000")